In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
df = pd.read_csv('Deep/chap09/data/diabetes.csv')
X = df[df.columns[:-1]]
y = df['Outcome']

X = X.values
y = torch.tensor(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [29]:
ms = MinMaxScaler()
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = ms.fit_transform(y_train)
y_test = ms.fit_transform(y_test)

In [30]:
class customdataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

In [31]:
train_data = customdataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = customdataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

train_loader = DataLoader(dataset = train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=64, shuffle=False)

In [32]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(8, 64, bias=True)
        self.layer_2 = nn.Linear(64, 64, bias=True)
        self.layer_out = nn.Linear(64, 1, bias=True)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        X = self.relu(self.layer_1(inputs))
        X = self.batchnorm1(X)
        X = self.relu(self.layer_2(X))
        X = self.batchnorm2(X)
        X = self.dropout(X)
        X = self.layer_out(X)
        return X

In [33]:
epochs = 1000+1
print_epoch = 100
LEARNING_RATE = 1e-2

model = binaryClassification()
model.to(device)
print(model)
BCE = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

binaryClassification(
  (layer_1): Linear(in_features=8, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [34]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc*100)
    return acc

In [35]:
for epoch in range(epochs):
    iteration_loss = 0.
    iteration_accuracy = 0.

    model.train()
    for i, data in enumerate(train_loader):
        X, y = data
        y_pred = model(X.float())
        loss = BCE(y_pred, y.reshape(-1, 1).float())

        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch % print_epoch == 0):
        print('Train: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch, iteration_loss/(i+1), iteration_accuracy/(i+1)))

    iteration_loss = 0.
    iteration_accuracy = 0.
    model.eval()
    for i, data in enumerate(test_loader):

        X, y = data
        y_pred = model(X.float())
        loss = BCE(y_pred, y.reshape(-1, 1).float())
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
    if(epoch % print_epoch == 0):
        print('Test: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch, iteration_loss/(i+1), iteration_accuracy/(i+1)))

Train: epoch: 0 - loss: 0.69351; acc: 56.900
Test: epoch: 0 - loss: 0.69451; acc: 57.333
Train: epoch: 100 - loss: 0.35883; acc: 84.600
Test: epoch: 100 - loss: 0.54734; acc: 75.333
Train: epoch: 200 - loss: 0.26639; acc: 89.000
Test: epoch: 200 - loss: 0.63916; acc: 73.667
Train: epoch: 300 - loss: 0.18723; acc: 92.500
Test: epoch: 300 - loss: 0.72079; acc: 74.000
Train: epoch: 400 - loss: 0.13430; acc: 95.300
Test: epoch: 400 - loss: 0.85671; acc: 74.667
Train: epoch: 500 - loss: 0.10167; acc: 96.500
Test: epoch: 500 - loss: 0.95871; acc: 75.000
Train: epoch: 600 - loss: 0.08819; acc: 97.000
Test: epoch: 600 - loss: 0.99086; acc: 71.333
Train: epoch: 700 - loss: 0.04564; acc: 98.800
Test: epoch: 700 - loss: 1.17641; acc: 71.667
Train: epoch: 800 - loss: 0.06572; acc: 97.800
Test: epoch: 800 - loss: 1.21188; acc: 68.333
Train: epoch: 900 - loss: 0.06063; acc: 98.600
Test: epoch: 900 - loss: 1.36416; acc: 66.667
Train: epoch: 1000 - loss: 0.06231; acc: 97.300
Test: epoch: 1000 - loss: 